In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
import datetime

In [9]:
def mg(df, population):
    return pd.merge(df, population, how = 'inner', on= 'Country')

def formatData(path):
    df = pd.read_csv(path, index_col = 'Country/Region') #Nombre de país como índice
    df.index.name = 'Country'
    df.drop(['Lat', 'Long'], axis = 1 ,inplace = True)
    col_list = [df.columns[0]] #provincias
    sum_days = {}                                            
    for i in range(1,df.shape[1],1):
        col_list.append(pd.Timestamp(df.columns[i]).date())
        sum_days[col_list[i]] = np.sum
    df.columns = col_list
    df = df.groupby('Country').aggregate(sum_days)
    return df

def acumulados(data, country, tipo, f):
    fecha = data.loc[country].index
    fecha = fecha[0:len(fecha)-1]
    casos = data.loc[country].values
    casos = casos[0:len(casos)-1]
    fecha = pd.to_datetime(fecha) 
    casos = pd.Series(casos, index=fecha)
    if tipo == 'Absolutos':
        if f == 'Semanal':
            casos = casos.groupby(pd.Grouper(freq='W-MON')).sum()
            return(casos.index, casos.values)
        else:
            return(fecha, casos)
    else:
        poblacion = data.loc[country, 'Population (2020)']
        casos_tasa = casos / poblacion * 100000  
        if f == 'Semanal':
            casos_tasa = casos_tasa.groupby(pd.Grouper(freq='W-MON')).sum()
            return(casos_tasa.index, casos_tasa.values)
        else:
            return(fecha, casos_tasa)

def nuevos(data, country, tipo,f):
    casos = data.drop(['Population (2020)'], axis=1)
    casos_pais = casos.loc[country]
    casos_nuevos = casos_pais.diff().fillna(casos_pais.iloc[0])
    casos_nuevos = casos_nuevos.clip(lower=0) 
    if tipo == 'Absolutos':
        fecha = casos_nuevos.index
        fecha = pd.to_datetime(fecha) 
        casos_nuevos = pd.Series(casos_nuevos.values, index=fecha)
        if f == 'Diario':
            return (fecha, casos_nuevos)
        else:
            #casos_nuevos = casos_nuevos.groupby(pd.Grouper(freq='W-MON')).sum()
            return(fecha, casos_nuevos)
    else:
        poblacion = data.loc[country, 'Population (2020)']
        casos_tasa = casos_nuevos / poblacion * 100000
        fecha = casos_tasa.index
        return (fecha, casos_tasa.values)
    
def generate_plot(data, c1, c2, caso, titulo, escala, frecuencia):
    fig, ax = plt.subplots(figsize=(8, 5), tight_layout=True)
    
    if caso == "Acumulados":
        (c1_x, c1_y) = acumulados(data, c1, escala, frecuencia)
        (c2_x, c2_y) = acumulados(data, c2, escala, frecuencia)               
        ax.set(xlabel="Fecha", ylabel=f"Casos acumulados por {escala}")
    else:
        (c1_x, c1_y) = nuevos(data, c1, escala, frecuencia)
        (c2_x, c2_y) = nuevos(data, c2, escala, frecuencia)
        ax.set(xlabel="Fecha", ylabel=f"Casos nuevos por {escala}")
                        
    ax.plot(c1_x, c1_y, label=c1); ax.legend()
    ax.plot(c2_x, c2_y ,label=c2); ax.legend()
    ax.set_title(titulo, fontsize=18)
    plt.show()

def plot_confirmados(c1, c2, caso, escala, frecuencia):
    generate_plot(confirmados, c1, c2, caso, "Confirmados", escala, frecuencia)
    
def plot_recuperados(c1, c2, caso, escala, frecuencia):
    generate_plot(recuperados, c1, c2, caso, "Recuperados", escala, frecuencia)

def plot_decesos(c1, c2, caso, escala, frecuencia):
    generate_plot(muertes, c1, c2, caso, "Decesos", escala, frecuencia)
    
def action_1():
    widgets.interact(plot_confirmados, c1=c1_dropdown, c2=c2_dropdown, caso=casos_dropdown, escala=escala_dropdown, frecuencia=frecuencia_dropdown)

def action_2():
    widgets.interact(plot_recuperados, c1=c1_dropdown, c2=c2_dropdown, caso=casos_dropdown, escala=escala_dropdown, frecuencia=frecuencia_dropdown)

def action_3():
    widgets.interact(plot_decesos, c1=c1_dropdown, c2=c2_dropdown, caso=casos_dropdown, escala=escala_dropdown, frecuencia=frecuencia_dropdown)

In [10]:
pc = {'Burma': 'Myanmar', 'Korea, South': 'South Korea', 'Taiwan*': 'Taiwan','US':'United States',
     'Saint Kitts and Nevis': 'Saint Kitts & Nevis', 'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
     'Sao Tome and Principe': 'Sao Tome & Principe'}
pp = {'Czech Republic (Czechia)': 'Czechia','Congo': 'Congo (Brazzaville)','DR Congo': 'Congo (Kinshasa)'}

In [11]:
confirmados = formatData('data/covid19_confirmados.csv')

population = pd.read_csv('data/poblacion_mundial2020.csv', index_col='Country (or dependency)')
population.index.name = 'Country'

population.drop(['Yearly Change', 'Net Change','Density (P/Km²)','Land Area (Km²)',
                 'Migrants (net)', 'Fert. Rate', 'Med. Age','Urban Pop %', 'World Share'], 
                  axis = 1 ,inplace = True)
p_confirmados = set(confirmados.index.values)
p_population = set(population.index.values)

confirmados.index = confirmados.index.to_series().replace(pc)
population.index = population.index.to_series().replace(pp)

confirmados = mg(confirmados, population)

recuperados = formatData('data/covid19_recuperados.csv')
recuperados.index = recuperados.index.to_series().replace(pc)
recuperados = mg(recuperados, population)

muertes = formatData('data/covid19_muertes.csv')
muertes.index = muertes.index.to_series().replace(pc)
muertes = mg(muertes, population)

In [12]:
OCDE = ['Austria', 'Australia', 'Belgium', 'Canada', 'Chile', 'Colombia', 'Denmark', 'Estonia',
        'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy',
        'Japan', 'Latvia', 'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand', 'Norway',
        'Poland', 'Portugal', 'Slovenia', 'Spain', 'Sweden',
        'Switzerland', 'Turkey', 'United States', 'United Kingdom', 'Slovakia', 'Czechia']

In [13]:
c1_dropdown = widgets.Dropdown(options=list(OCDE), description='País:', disabled=False)
c2_dropdown = widgets.Dropdown(options=list(OCDE), description='País:', disabled=False)
casos_dropdown = widgets.Dropdown(description="Casos:", options=["Nuevos", "Acumulados"])
escala_dropdown = widgets.Dropdown(description="Escala:", options=["Absolutos", "Relativos"])
frecuencia_dropdown = widgets.Dropdown(description="Frecuencia:", options=["Diarios", "Semanal"])

out1 = widgets.interactive(action_1)
out2 = widgets.interactive(action_2)
out3 = widgets.interactive(action_3)

tab  = widgets.Tab(children=[out1, out2, out3])
tab.set_title(0, 'Confirmados')
tab.set_title(1, 'Recuperados')
tab.set_title(2, 'Muertes')

display(tab)